Source: https://github.com/quantumlib/Cirq/blob/master/examples/bernstein_vazirani.py

In [1]:
import random
import cirq

In [3]:
#creating an oracle
#this implements the function f(a) = a*factors + bias(mod 2)

def make_oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit):

    if secret_bias_bit:
        yield cirq.X(output_qubit)
    
    for qubit, bit in zip(input_qubits, secret_factor_bits):
        if bit:
            yield cirq.CNOT(qubit, output_qubit)

In [4]:
#create the Bernstein-Vazirani algorithm
#this solves for factors in f(a) = a*factors + bias(mod 2) with one query

def make_bernstein_vazirani(input_qubits, output_qubit, oracle):
        #instantiate circuit, initialize qubits
        c = cirq.Circuit()
        c.append([ cirq.X(output_qubit),
                   cirq.H(output_qubit),
                   cirq.H.on_each(*input_qubits)])
        
        # query oracle, measure in X basis
        c.append(oracle)
        c.append([cirq.H.on_each(*input_qubits), 
                  cirq.measure(*input_qubits,key='result')])
        
        return c

In [6]:
#defining the bitstring

def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

In [57]:
def main(qubit_count = 8):
    circuit_sample_count = 3
    
    #choose qubits to use
    input_qubits = [ cirq.GridQubit(i,0) for i in range(qubit_count) ]
    output_qubit = cirq.GridQubit(qubit_count, 0)
    
    #pick coefficients for the oracle, create circuit to query it
    secret_bias_bit = random.randint(0,1)
    secret_factor_bits = [ random.randint(0,1) for i in range(qubit_count) ]
    oracle = make_oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit)
    
    print(f"Secret function:\n{', '.join(str(e) for e in secret_factor_bits)}\n")
    print(f"Secret bias bit:\n{secret_bias_bit}\n")
    
    #embed oracle into a quantum circuit, query it exactly once
    circuit = make_bernstein_vazirani(input_qubits, output_qubit, oracle)
    print(f"Circuit:\n{circuit}\n")
    
    #sample from the circuit a few times
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions = circuit_sample_count)
    frequencies = result.histogram(key='result', fold_func = bitstring)
    print(f"Sampled results:\n{frequencies}\n")
    
    #check whether we actually found the secret value
    most_common_bitstring = frequencies.most_common(1)[0][0]
    print(f"Most common bitstring matching secret factors?\n{most_common_bitstring == bitstring(secret_factor_bits)}")

In [61]:
main()

Secret function:
1, 0, 0, 1, 0, 0, 1, 0

Secret bias bit:
1

Circuit:
(0, 0): ───H───────────@───H───────────M('result')───
                       │               │
(1, 0): ───H───H───────┼───────────────M─────────────
                       │               │
(2, 0): ───H───H───────┼───────────────M─────────────
                       │               │
(3, 0): ───H───────────┼───@───H───────M─────────────
                       │   │           │
(4, 0): ───H───H───────┼───┼───────────M─────────────
                       │   │           │
(5, 0): ───H───H───────┼───┼───────────M─────────────
                       │   │           │
(6, 0): ───H───────────┼───┼───@───H───M─────────────
                       │   │   │       │
(7, 0): ───H───H───────┼───┼───┼───────M─────────────
                       │   │   │
(8, 0): ───X───H───X───X───X───X─────────────────────

Sampled results:
Counter({'10010010': 3})

Most common bitstring matching secret factors?
True


In [12]:
qubit_count = 8

In [40]:
circuit_sample_count = 10

In [41]:
#choose qubits to use
input_qubits = [ cirq.GridQubit(i,0) for i in range(qubit_count) ]
output_qubit = cirq.GridQubit(qubit_count, 0)

In [42]:
#pick coefficients for the oracle, create circuit to query it
secret_bias_bit = random.randint(0,1)
secret_factor_bits = [ random.randint(0,1) for i in range(qubit_count) ]

In [43]:
oracle = make_oracle(input_qubits, output_qubit, secret_factor_bits, secret_bias_bit)

In [44]:
', '.join(str(e) for e in secret_factor_bits)

'0, 1, 0, 0, 0, 0, 1, 1'

In [45]:
#embed the oracle into a special quantum circuit, querying it exactly once

circuit = make_bernstein_vazirani(input_qubits, output_qubit, oracle)
circuit

(0, 0): ───H───H───────────────────────M('result')───
                                       │
(1, 0): ───H───────────@───H───────────M─────────────
                       │               │
(2, 0): ───H───H───────┼───────────────M─────────────
                       │               │
(3, 0): ───H───H───────┼───────────────M─────────────
                       │               │
(4, 0): ───H───H───────┼───────────────M─────────────
                       │               │
(5, 0): ───H───H───────┼───────────────M─────────────
                       │               │
(6, 0): ───H───────────┼───@───H───────M─────────────
                       │   │           │
(7, 0): ───H───────────┼───┼───@───H───M─────────────
                       │   │   │
(8, 0): ───X───H───X───X───X───X─────────────────────

In [46]:
#simulate and sample the circuit multiple times
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions = circuit_sample_count)
frequencies = result.histogram(key = 'result', fold_func = bitstring)

frequencies

Counter({'01000011': 10})

In [47]:
#check whether or not we found the secret value

most_common_bitstring = frequencies.most_common(1)[0][0]

most_common_bitstring

'01000011'

In [48]:
frequencies.most_common(1)

[('01000011', 10)]